In [1]:
import os
import pandas as pd
import numpy as np

os.chdir("C:\\Users\\StuartWilson\\Documents\\Portfolio\\NEISS")

In [2]:
neiss = pd.read_csv('Data\\neiss.csv')
neiss = neiss.sample(frac=1)
print(neiss.shape)
print(neiss.columns)

c:\Users\StuartWilson\Documents\Portfolio\NEISS\Python\.venv\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (11,12,13,25) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


(1841588, 27)
Index(['Unnamed: 0', 'CPSC_Case_Number', 'Treatment_Date', 'Age', 'Sex',
       'Race', 'Other_Race', 'Hispanic', 'Body_Part', 'Diagnosis',
       'Other_Diagnosis', 'Body_Part_2', 'Diagnosis_2', 'Other_Diagnosis_2',
       'Disposition', 'Location', 'Fire_Involvement', 'Alcohol', 'Drug',
       'Narrative', 'Stratum', 'PSU', 'Weight', 'Product', 'Product_2',
       'Product_3', 'Year'],
      dtype='object')


In [3]:
#show how many 'not recorded' in location column
neiss['Location'].value_counts(normalize=True)

Home                             0.439636
Not recorded                     0.283534
Place of recreation or sports    0.130473
Other public property            0.070278
School/Daycare                   0.053731
Street or highway                0.021715
Farm/ranch                       0.000347
Mobile/Manufactured home         0.000212
Industrial                       0.000074
Name: Location, dtype: float64

In [4]:
neiss['Narrative'].head(3)


479165     15YOM PT COMPLAINING OF INJURING FOREHEAD AT F...
572400     15 YO F C/O CHEST PAIN FOR 1 DAY STATES SHE WA...
1642097    33 YOM WAS USING A SAW AT HOME WHEN IT SLIPPED...
Name: Narrative, dtype: object

In [5]:
#remove unnecessary columns
neiss_clf = neiss[['Location', 'Narrative']]
neiss_clf.head()

,Location,Narrative
479165,Place of recreation or sports,15YOM PT COMPLAINING OF INJURING FOREHEAD AT F...
572400,Not recorded,15 YO F C/O CHEST PAIN FOR 1 DAY STATES SHE WA...
1642097,Home,33 YOM WAS USING A SAW AT HOME WHEN IT SLIPPED...
663151,Not recorded,57YOM TRIPPED & FELL FROM 1 STEP TODAY DX: RIG...
1214985,Not recorded,14YOF HAD IN TOENAIL ATTEMPT TO CUT/TRIM (IMPL...


In [6]:
#show value counts for locations after removing 'not recorded'
neiss_clf = neiss_clf.loc[~neiss['Location'].isin(['Not recorded', 'Farm/ranch',
                                           'Mobile/Manufactured home', 'Industrial']) == True]


In [7]:
#create indicator variables for locations

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(list(neiss_clf['Location'].unique()))
neiss_clf['Location_label'] = le.transform(neiss_clf['Location'])


In [8]:
# clean strings
# get rid of non alpha numeric
# fix the yof and yom
neiss_clf['Text'] = neiss_clf['Narrative'].str.replace(r'[^a-zA-Z0-9 ]+', ' ').str.lower()
neiss_clf['Text'] = neiss_clf['Text'].str.replace('yof', ' yof ')
neiss_clf['Text'] = neiss_clf['Text'].str.replace('yom', ' yom ')
neiss_clf['Text'] = neiss_clf['Text'].str.replace('y o f', ' yof ')
neiss_clf['Text'] = neiss_clf['Text'].str.replace('y o m', ' yom ')
neiss_clf['Text'] = neiss_clf['Text'].str.replace('yf', ' yof ')
neiss_clf['Text'] = neiss_clf['Text'].str.replace('ym', ' yom ')
neiss_clf['Text'] = neiss_clf['Text'].str.replace('yo f', ' yof ')
neiss_clf['Text'] = neiss_clf['Text'].str.replace('yo m', ' yom ')
neiss_clf['Text'] = neiss_clf['Text'].str.replace('yr', ' yr ')
neiss_clf['Text'] = neiss_clf['Text'].str.replace('dx', ' dx ')
neiss_clf['Text'] = neiss_clf['Text'].str.replace('sx', ' sx ')
neiss_clf['Text'] = neiss_clf['Text'].str.replace('tx', ' tx ')
neiss_clf['Text'] = neiss_clf['Text'].str.replace('fx', ' fx ')
neiss_clf['Text'] = neiss_clf['Text'].str.replace('hx', ' hx ')
neiss_clf['Text'] = neiss_clf['Text'].str.replace(' wa s', ' was ')
neiss_clf['Text'] = neiss_clf['Text'].str.replace(' wa ', ' was ')


neiss_clf['Text'].head()

<ipython-input-8-d2ca7e906624>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  neiss_clf['Text'] = neiss_clf['Narrative'].str.replace(r'[^a-zA-Z0-9 ]+', ' ').str.lower()


479165     15 yom  pt complaining of injuring forehead at...
1642097    33  yom  was using a saw at home when it slipp...
1290283    15  yom  wrestling  developed chest pain   dx ...
847392       14  yom  playing basketball    dx  ankle sprain
1626482    8  yom    scalp laceration   pt was playing at...
Name: Text, dtype: object

In [9]:
#tokenize narrative
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
neiss_clf["Tokens"] = neiss_clf["Text"].apply(tokenizer.tokenize)


In [10]:
from tabulate import tabulate

table =  neiss_clf[612:615]
header = table.columns
print(tabulate(table, header, tablefmt="pipe"))

|         | Location          | Narrative                                                                                                                                     |   Location_label | Text                                                                                                                                               | Tokens                                                                                                                                                                                                          |
|--------:|:------------------|:----------------------------------------------------------------------------------------------------------------------------------------------|-----------------:|:---------------------------------------------------------------------------------------------------------------------------------------------------|:------------------------------------------------------------------------------------------------------

In [11]:
#take sample to save time fitting model
sample = neiss_clf.sample(500000)
sample['Location'].value_counts(normalize=True)

Home                             0.614104
Place of recreation or sports    0.182156
Other public property            0.098280
School/Daycare                   0.074990
Street or highway                0.030470
Name: Location, dtype: float64

In [12]:
#bag of words
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
    

def bow(token, data=sample):
    text = token.apply(lambda x: ' '.join(x))
    list_corpus = text.tolist()
    list_labels = data['Location_label'].tolist()
    X_train, X_test, y_train, y_test = train_test_split(list_corpus, list_labels, random_state=42,
                                                            test_size=0.2)
    count_vect = CountVectorizer()

    X_train_counts = count_vect.fit_transform(X_train)
    X_val_counts = count_vect.transform(X_val)
    return X_train, X_val, y_train, y_val, X_test, y_test, X_train_counts, X_val_counts

X_train, X_val, y_train, y_val, X_test, y_test, X_train_counts, X_val_counts = bow(sample['Tokens'])

clf = SGDClassifier(random_state=42)
clf.fit(X_train_counts, y_train)
y_pred = clf.predict(X_val_counts)

def get_metrics(y_test, y_pred):  
    precision = precision_score(y_test, y_pred, pos_label=None,
                                    average='weighted')             
    recall = recall_score(y_test, y_pred, pos_label=None,
                              average='weighted')
    f1 = f1_score(y_test, y_pred, pos_label=None, average='weighted')
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy, precision, recall, f1

accuracy, precision, recall, f1 = get_metrics(y_val, y_pred)
print("accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy, precision, recall, f1))

accuracy = 0.903, precision = 0.906, recall = 0.903, f1 = 0.898
